- This notebook cleans data obtained from public, online sources
- This data, particularly the census data, was not in an easily accessible format for Tableau visualization or manipulation

In [1]:
import pandas as pd
import re

## President Votes by State Cleaning
[Presidential Votes by State Link](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX#)

In [3]:
pres = pd.read_csv('1976-2020-president.csv')

In [4]:
pres2020 = pres[pres['year']==2020]

In [8]:
pres2020 = pres2020[(pres2020['party_simplified']=='REPUBLICAN') | (pres2020['party_simplified']=='DEMOCRAT')]

In [10]:
pres2020dem = pres2020[pres2020['party_simplified']=='DEMOCRAT'][['state', 'candidatevotes', 'totalvotes']]

In [12]:
pres2020dem['percent_democrat'] = round((pres2020dem.candidatevotes / pres2020dem.totalvotes)*100, 2)

In [14]:
pres2020rep = pres2020[pres2020['party_simplified']=='REPUBLICAN'][['state', 'candidatevotes', 'totalvotes']]

In [16]:
pres2020rep['percent_republican'] = round((pres2020rep.candidatevotes / pres2020rep.totalvotes)*100, 2)

In [18]:
presvotes = pres2020dem.merge(pres2020rep, left_on = 'state', right_on = 'state', how='inner') \
[['state', 'percent_democrat', 'percent_republican']]

In [20]:
presvotes.head()

,state,percent_democrat,percent_republican
0,ALABAMA,36.57,62.03
1,ALASKA,42.77,52.83
2,ARIZONA,49.36,49.06
3,ARKANSAS,34.78,62.40
4,CALIFORNIA,63.48,34.32


In [343]:
presvotes.to_csv('votes_states.csv', index=False)

## Education Demographics by State Cleaning

[Education Demographics Census Link](https://data.census.gov/table/ACSST5Y2020.S1501?t=Educational%20Attainment&g=010XX00US$0400000&moe=false&tp=true!)
 - Have to transpose data in link, then only select only "Population 25 years and over" column set

In [99]:
ed = pd.read_csv('EducationDemographics.csv')

In [101]:
ed = ed.drop(columns = ['AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])

In [107]:
i = 0
while i < len(ed)-4:
    for c in ed.columns:
        if c!='Label (Grouping)':
            ed[c][i] = ed[c][i+4]
    i+=13

In [113]:
ed = ed[(ed['Label (Grouping)'] != '\xa0\xa0\xa0\xa0Total') & 
        (ed['Label (Grouping)'] != '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Estimate') & 
        (ed['Label (Grouping)'] != 'Percent')]

In [52]:
pattern = re.compile(r'\xa0')

In [145]:
ed_filtered = ed[~ed['Label (Grouping)'].str.contains(pattern)]

In [153]:
ed_filtered = \
ed_filtered.rename(
    columns={'Label (Grouping)' : 'state',
            'AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade' : 'Less than 9th',
            'AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma' : 'Less than 12th',
            'AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)' : 'High school graduate',
            'AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree': 'Some college',
            "AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree" : "Associate's Degree", 
            "AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher": "Bachelor's or higher"})

In [161]:
ed_filtered = ed_filtered.reset_index(drop=True)

In [167]:
ed_filtered['state'] = ed_filtered['state'].str.upper()

In [171]:
for c in ed_filtered.columns:
    ed_filtered[c] = ed_filtered[c].str.replace("%", "")

In [175]:
ed_filtered.head(5)

,state,Less than 9th,Less than 12th,High school graduate,Some college,Associate's Degree,Bachelor's or higher
0,ALABAMA,4.00,9.10,30.30,21.60,8.70,26.20
1,ALASKA,2.40,4.50,28.40,26.00,8.70,30.00
2,ARIZONA,5.10,7.00,23.80,24.90,8.90,30.30
3,ARKANSAS,4.70,8.10,33.90,22.00,7.50,23.80
4,CALIFORNIA,8.90,7.20,20.40,20.90,8.00,34.70


In [341]:
ed_filtered.to_csv('education_states.csv', index=False)

## Race Demographics by State Cleaning

[Race Demographics Census Link](https://data.census.gov/table/ACSDP5Y2020.DP05?g=010XX00US$0400000&moe=false&tp=true)
- Have to select only the "White" column

In [42]:
race = pd.read_csv('RaceDemographics.csv')

In [44]:
race.head(5)

,Label (Grouping),RACE!!Total population!!One race!!White
0,Alabama,NaN
1,Estimate,"3,302,834"
2,Percent,67.50%
3,Alaska,NaN
4,Estimate,"466,961"


In [46]:
race.columns

Index(['Label (Grouping)', 'RACE!!Total population!!One race!!White'], dtype='object')

In [48]:
i = 0
while i < len(race)-2:
    race['RACE!!Total population!!One race!!White'][i] = race['RACE!!Total population!!One race!!White'][i+2]
    i+=3

In [54]:
race_filtered = race[~race['Label (Grouping)'].str.contains(pattern)]

In [56]:
race_filtered = race_filtered.reset_index(drop=True)

In [58]:
race_filtered = \
race_filtered.rename(
    columns = {'Label (Grouping)' : 'state',
               'RACE!!Total population!!One race!!White' : 'percent_white'})

In [60]:
race_filtered['percent_white'] = race_filtered['percent_white'].str.replace("%", "")

In [62]:
race_filtered['state'] = race_filtered['state'].str.upper()

In [64]:
race_filtered.head(5)

,state,percent_white
0,ALABAMA,67.50
1,ALASKA,63.40
2,ARIZONA,73.80
3,ARKANSAS,75.40
4,CALIFORNIA,56.10


In [66]:
race_filtered.to_csv('race_states.csv', index=False)

## Urban Demographics by State Cleaning

[Urban Demographics Census Link](https://data.census.gov/table/DECENNIALCD1182020.P2)

In [68]:
urban = pd.read_csv('UrbanDemographics.csv')

In [70]:
urban

,Label (Grouping),Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
0,Total:,"5,024,279","733,391","7,151,502","3,011,524","39,538,223","5,773,714","3,605,944","989,948","689,545",...,"6,910,840","29,145,505","3,271,616","643,077","8,631,393","7,705,281","1,793,716","5,893,718","576,851","3,285,874"
1,Urban,"2,900,880","475,967","6,385,230","1,670,677","37,259,490","4,966,936","3,110,153","817,817","689,545",...,"4,577,282","24,400,697","2,937,303","225,850","6,528,313","6,424,035","800,857","3,953,691","357,750","3,018,908"
2,Rural,"2,123,399","257,424","766,272","1,340,847","2,278,733","806,778","495,791","172,131",0,...,"2,333,558","4,744,808","334,313","417,227","2,103,080","1,281,246","992,859","1,940,027","219,101","266,966"
3,Not defined for this file,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
urban = urban.melt(id_vars = 'Label (Grouping)', value_vars = urban.columns[1:])

In [74]:
totals = urban[(urban['Label (Grouping)'] == 'Total:')]

In [76]:
urb = urban[(urban['Label (Grouping)'] == '\xa0\xa0\xa0\xa0Urban')]

In [78]:
rur = urban[(urban['Label (Grouping)'] == '\xa0\xa0\xa0\xa0Rural')]

In [80]:
urb = urb.rename(columns={'value': 'urban_pop', 'variable':'state'}) 

In [82]:
rur = rur.rename(columns={'value': 'rural_pop', 'variable':'state'}) 

In [84]:
totals = totals.rename(columns={'value': 'total_pop', 'variable':'state'}) 

In [86]:
merged = urb.merge(rur, left_on='state', right_on='state', how='inner') \
    .merge(totals, left_on='state', right_on = 'state', how='inner')[['state', 'urban_pop', 'rural_pop', 'total_pop']]

In [88]:
merged['urban_pop'] = merged['urban_pop'].str.replace(",", "")

In [90]:
merged['rural_pop'] = merged['rural_pop'].str.replace(",", "")

In [92]:
merged['total_pop'] = merged['total_pop'].str.replace(",", "")

In [94]:
merged['urban_pop'] = merged['urban_pop'].astype(int)

In [96]:
merged['rural_pop'] = merged['rural_pop'].astype(int)

In [98]:
merged['total_pop'] = merged['total_pop'].astype(int)

In [100]:
merged['urban_percent'] = round((merged['urban_pop'] / merged['total_pop']) * 100, 2)

In [102]:
merged['rural_percent'] = round((merged['rural_pop'] / merged['total_pop']) * 100, 2)

In [106]:
merged['state'] = merged['state'].str.upper()

In [108]:
merged.head()

,state,urban_pop,rural_pop,total_pop,urban_percent,rural_percent
0,ALABAMA,2900880,2123399,5024279,57.74,42.26
1,ALASKA,475967,257424,733391,64.90,35.10
2,ARIZONA,6385230,766272,7151502,89.29,10.71
3,ARKANSAS,1670677,1340847,3011524,55.48,44.52
4,CALIFORNIA,37259490,2278733,39538223,94.24,5.76


In [110]:
merged.to_csv('urban_states.csv', index=False)